# Week 9 Extra Credit: Basic IR System

## Basic Text preprocessing steps
removing noise: anything that isn’t a standard number or letter
removing stop words: very common words that add little value in analysis are removed from the vocabulary.
stemming: reducing inflected (or derived) words to their stem, base or root form 
lemmatization: similar to stemming, however stemming can often create non-words, whereas lemmas are actual words

## Bag of Words (BoW) Model
After preprocessing, text needs to be transformed into a meaningful number vectors for use in ML algorithms. The BoW model represents text as a matrix of word counts within a document. It's called a “bag of words" because information about the order or structure of words is discarded. The model only cares whether the known words occur in the document, but not where they occur. Intuitively, documents are similar if they have similar content.
It involves:
 - a vocabulary of known words
 - a measure of the presence of known words

For example, given a dictionary containing {Learning, is, the, not, great}, to vectorize the text “Learning is great”.
Its vector representation would be : $(1, 1, 0, 0, 1)$, where the numbers represent their word counts.

## TF-IDF
With BoW, highly frequent words start to dominate the document, but such words may not contain much informational content. It also gives more weight to longer documents than shorter documents.  

One approach is to rescale the frequency of words by how often they appear in all documents. The scores for frequent words that are also frequent across all documents are penalized. This scoring is called Term Frequency-Inverse Document Frequency, where
 - Term Frequency: a scoring of the frequency of the word in the current document
    - TF = (Number of times term t appears in a document)/(Number of terms in the document)

 - Inverse Document Frequency: a scoring of how rare a word is across documents.
    - IDF = $1+log(N/n)$, where, $N$ is the number of documents and n is the number of documents a term $t$ has appeared in.

## Cosine Similarity
A measure of similarity between two non-zero vectors of an inner product space</ul>
 - Tf-idf weight is a weight often used in information retrieval (IR) and text mining.
 - It is a statistical measure used to evaluate how important a word is to a document in a collection or corpus
     - Cosine Similarity $(d1, d2)= Dot product (d1, d2) / ||d1|| * ||d2||$ where $d1,d2$ are two non zero vectors.
     
**Reference:**  
 1. https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
 1. https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
 1. https://scikit-learn.org/stable/modules/metrics.html#cosine-similarity
 1. http://jonathansoma.com/lede/foundations/classes/text%20processing/tf-idf/

In [10]:
import io
import random
import string
import re
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [11]:
docs = [
    "Trump became president after winning the political election. \
    Though he lost the support of some republican friends, Trump is friends with President Putin",

    "President Trump says Putin had no political interference in the election outcome. \
    He says it was a witchhunt by political parties. He claimed President Putin is a friend who had nothing \
    to do with the election",
    
    "The COVID-19 pandemic continues to ravage populations around the globe. Worldwide over 2.5 million have died. \
    In the United States the death toll is over half a million",

    "Post elections, Vladimir Putin became President of Russia. \
    President Putin had served as the Prime Minister earlier in his political career",
    
    "Experts point out that the COVID-19 vaccines have nothing to do with the reproductive system, and that the vaccines \
    do not use live viruses or alter human DNA.",
    
    "President Joe Biden said Tuesday that the U.S. expects to take delivery of enough coronavirus vaccines for\
    all adult Americans by the end of May, two months earlier than anticipated, as his administration \
    announced that drugmaker Merck & Co. will help produce rival Johnson & Johnson’s newly approved shot."
]

Text preprocessing helper functions which will be passed as a parameter to or Vectorization object.
 - removes punctionation and special characters
 - lower-cases
 - stemming or lemmatization; in this case I'll choose lemmatization as stemming can result in 'non-words'

## Find base form of a word (eg: stemming or lemmatization)

In [12]:
wn_lemmer = WordNetLemmatizer()
ps_stemmer = PorterStemmer()

#  custom tokenizer
def custom_tokenizer(str_input):
   
    # remove special characters and stem
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    
    #words = [ps_stemmer.stem(word) for word in words]    # stemmer
    words = [wn_lemmer.lemmatize(word) for word in words] # lemmatizer
    
    return words

To compute the cosine similarity, you need the word count of the words in each document. We will use TF-IDF
 - With TfidfVectorizer or CountVectorizer, a tokenizer can be specified that allows for custom preprocessing of the words.

## First let's look at CountVectorizer

In [13]:
countVec = CountVectorizer(stop_words='english', tokenizer=custom_tokenizer)
X = countVec.fit_transform(docs)

# display as a dataframe
df = pd.DataFrame(X.toarray(), columns=countVec.get_feature_names())
df

,2,5,administration,adult,alter,american,announced,anticipated,approved,biden,...,u,united,use,vaccine,virus,vladimir,wa,winning,witchhunt,worldwide
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,1,2,1,0,0,0,0,0
5,0,0,1,1,0,1,1,1,1,1,...,1,0,0,1,0,0,0,0,0,0


We can see vaccine appeared 2X in doc 5 (eg: index by 0) and 1X in doc 6.  

doc 5: "Experts point out that the COVID-19 vaccines have nothing to do with the reproductive system, and that the vaccines do not use live viruses or alter human DNA."  

We can also see 2 and 5 appearing in doc 3: "The COVID-19 pandemic continues to ravage populations around the globe. Worldwide over 2.5 million have died. In the United States the death toll is over half a million"  

Due to remval of punctuation during preprocessing, 2.5 was transformed into 2 and 5. This is an example of some of the challenges of preprocessing. Similarly, in doc 6, U.S. is transformed to u and s. In this case, we need a more sophisticated regular expression to handle this scenario.  

## Next lets use use TF-IDF which account how often a term shows up in determining importance

In [5]:
TfidfVec = TfidfVectorizer(stop_words='english',        # remove stop words
                           tokenizer=custom_tokenizer   # preprocessing
                          )
tfidf = TfidfVec.fit_transform(docs)


# display as a dataframe
df = pd.DataFrame(tfidf.toarray(), columns=TfidfVec.get_feature_names())
df

,2,5,administration,adult,alter,american,announced,anticipated,approved,biden,...,u,united,use,vaccine,virus,vladimir,wa,winning,witchhunt,worldwide
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.286005,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.232469,0.000000,0.232469,0.000000
2,0.231419,0.231419,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.231419,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.231419
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.289205,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.284416,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.284416,0.466450,0.284416,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.174883,0.174883,0.000000,0.174883,0.174883,0.174883,0.174883,0.174883,...,0.174883,0.000000,0.000000,0.143406,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Instead of just being a count of words, it’s the percentage of the words.  

Looking again at doc 5: "Experts point out that the COVID-19 vaccines have nothing to do with the reproductive system, and that the vaccines do not use live viruses or alter human DNA."  

We can see `vacine` represents `46.6%` , `use` represents `28%` and `virus` is `28%`

## Cosine similarity
Let's calculate similarity of documents using cosine_similarity, given our vectorized corpus. Suppose we had a question whose answer is listed in our corpus. This is a simplied example of IR (information retrieval)

1. first add the query to our corpus
1. then compare the similarity of the query to the existing corpus, using cosine_similarity function
1. then return the document that is most similar to our query

In [6]:
# our query
newDoc = 'Who is Putin?'

# step 1
docs.append(newDoc)  # add our query to our corpus

# step 2
# need to re-vectorize given query addition
TfidfVec = TfidfVectorizer(stop_words='english', tokenizer=custom_tokenizer)
tfidf = TfidfVec.fit_transform(docs)
    
# compare the similarity of the query to the existing corpus    
vals = cosine_similarity(tfidf[-1],  # query. (eg: the last doc via append
                             tfidf)  # all documents

vals

array([[0.17482074, 0.28689806, 0.        , 0.35970216, 0.        ,
        0.        , 1.        ]])

Let's look at this closer
 - cosine_similarity compares consine simiarlity of 2 docs
 - cosine similarity of 2 documents will range from 0 to 1  

The following looks at the cosine similarity between our query and each of the docs

In [7]:
# this includes our query
print('docs shape:', tfidf.shape, '\n')  

# original docs
d1, d2, d3, d4, d5, d6 = tfidf[0], tfidf[1], tfidf[2], tfidf[3], tfidf[4], tfidf[5]

# our query
q = tfidf[6]

print('similarity between the query and existing docs')
print(f"{cosine_similarity(q,d1)}\n{cosine_similarity(q,d2)}\n{cosine_similarity(q,d3)}\n{cosine_similarity(q, d4)}\n{cosine_similarity(q,d5)}\n{cosine_similarity(q,d6)}")
     

docs shape: (7, 76) 

similarity between the query and existing docs
[[0.17482074]]
[[0.28689806]]
[[0.]]
[[0.35970216]]
[[0.]]
[[0.]]


We can see that doc 4 at index 3 is the most similar.  

The following describes how to pull out that index from the cosine similarity matrix

In [8]:
print('query', newDoc, '\n') # remind ourselves of the query

print(vals.argsort())       # returns the indexes for the values in sorted order
print(vals.argsort().shape) # it's dimension/shape

print()
print(vals.argsort()[0])     # get a list of indexes of the sorted array

idx = vals.argsort()[0][-2]  # get the index of the highest value (note: -2 is the last doc, since -1 is the query
print('index:', idx)

print()
flat = vals.flatten()        # flatten the array
flat.sort()
print(flat.shape)

print()
req_tfidf = flat[-2]        # pull out the similarity value
print(req_tfidf)

# show the doc that is most similar to our query
print()
if(req_tfidf==0):
    print("We didn't find any matches, please try again.")
else:
    print( docs[idx])
    docs.remove(newDoc) # need to remove the query

query Who is Putin? 

[[2 4 5 0 1 3 6]]
(1, 7)

[2 4 5 0 1 3 6]
index: 3

(7,)

0.3597021557041277

Post elections, Vladimir Putin became President of Russia.     President Putin had served as the Prime Minister earlier in his political career


Putting it all together to get the index of the document that is most similar to our queru

In [9]:
idx = vals.argsort()[0][-2]  # index

flat = vals.flatten()
flat.sort()

req_tfidf = flat[-2]
req_tfidf

0.3597021557041277